# Description

Data dictnionary

- id : o id do documento no Oasisbr
- oai_identifier_str : id externo que vem do repositório de origem
- instname_str : nome da instituição de origem
- instacron_str : acrônimo da instituição de origem
- network_acronym_str : acrônimo da rede de origem (pode descartar)
- network_name_str : nome da rede origem (pode descartar)
- title : título
- author : autores
- publishDate  : data da publicação
- format : tipo de documento (article, masterThesis, doctoralThesis, …)
- description : abstract
- topic : keywords
- language : idioma do documento
- eu_rights_str_mv : tipo de acesso (openAccess, closed, …)
- url : url externa para o documento

# Input Data

## json

In [ ]:
import pandas as pd
import json
from zipfile import ZipFile
import ijson #ijson-3.1.4

infile = '/mnt/d/ibict/oasisbr/oasisbr_1.zip'

In [ ]:
# import ujson #ujson-5.6.0
# with open(infile) as f:
#     file_contents = ujson.load(f)

metodo 0

In [ ]:
sel = []
with ZipFile(infile, 'r') as archive:
    file = archive.namelist()[0]
    with archive.open(file) as f:
        for record in ijson.items(f, "response.docs.item"):
            try:
                if record['instacron_str'] == "UEPB":
                    sel.append(record)
            except:
                pass
            break


In [ ]:
#  'title': 'Germination test and ornamental potential of different basil cultivars (Ocimum spp.)',
#  'author': ['França,Matheus Filipe de Moraes Sousa'],
#  'topic': ['Ocimum basilicum',
#   'Ocimum minimum',
#   'ornamental gardening',
#   'landscaping',
#   'ornamental edible plants'],
# 'url': ['http://www.scielo.br/scielo.php?script=sci_arttext&pid=S2447-536X2017000400385'],

In [ ]:
record

In [ ]:
# df = pd.DataFrame.from_dict(sel)
# df['format'].unique()
# df[df.format != ['masterThesis']]
# df.format.values[0]

metodo 1

# Blockchain Operations

In [1]:
import sys
import os

import requests
import json

import pandas as pd

from web3 import Web3

# SCRIPT_DIR = os.path.dirname(os.path.abspath('__file__'))
PROJECT_DIR = os.path.dirname(os.path.abspath(''))
sys.path.append(os.path.dirname(PROJECT_DIR))

from pyDpi.util import setup
from pyDpi.util.libs import invoke_contract_experimental

## enviroment setup

services init

In [ ]:
w3 = setup.load_blockchain_driver()
deployed_contracts = setup.load_deployed_smart_contracts(w3)
dpid_db = deployed_contracts['PidDB.sol']
epid_db = deployed_contracts['ExternalPidDB.sol']
auth_db = deployed_contracts['AuthoritiesDB.sol']

dpid_service = deployed_contracts['PIDService.sol']
epid_service = deployed_contracts['ExternalPIDService.sol']
sets_service = deployed_contracts['SearchTermService.sol']
auth_service = deployed_contracts['AuthoritiesService.sol']


chain_id,min_gas_price,pk = setup.get_exec_parameters()
account = w3.eth.account.privateKeyToAccount(pk)

variables

In [ ]:
nam = '8003'
ror_id = '3wf3'
addr = '0xf17f52151EbEF6C7334FAD080c5704D77216b732' 
dshoulder_prefix = 'fkwf'   

dnma setup

In [ ]:
recipt_tx, est_gas = invoke_contract_experimental(w3,account,chain_id, auth_service , 'create_dnam', ror_id , dshoulder_prefix, addr)
auth_id = recipt_tx['logs'][0]['topics'][1]
auth_id

In [ ]:
auth_db.caller.count_dnma()
Web3.toHex( auth_db.caller.get_dnma_by_index(0) ) # converte o indice 0 
msg = "DecentralizedNameMappingAuthority :: {} [ ror={} , shoulder_prefix={} ]".format(auth_db.caller.get_dnma(auth_id)[-1],
                                                                                        auth_db.caller.get_dnma(auth_id)[1],
                                                                                        auth_db.caller.get_dnma(auth_id)[1])

msg

configura o noid_provider da autoridade
 - nam ( ibict 8003)
 - addr da autho
 - noid_len : tamanho do noid
 - type (1 - dnma, 2 - sma)

In [ ]:
noid_len = 5

r_tx = invoke_contract(w3,account,chain_id, auth_service , 'configure_noid_provider_dnma', nam, auth_id , noid_len , 1)
noid_addr = r_tx['logs'][0]['topics'][1][12:]
noid_addr # endereco do contrato

## run

In [ ]:
addr =  "0xf17f52151EbEF6C7334FAD080c5704D77216b732" #from setup

create ark

In [ ]:
r_tx = invoke_contract_experimental(w3,account,chain_id, dpid_service , 'assingID', addr)
dark_id = r_tx['logs'][0]['topics'][1]
dark_id

In [ ]:
dpid_db.caller.get(dark_id)[1]

### create metadata

search term

In [ ]:
r_tx = invoke_contract(w3,account,chain_id, dpid_service, 'addSearchTerm',dark_id ,'DARK')
r_tx = invoke_contract(w3,account,chain_id, dpid_service, 'addSearchTerm',dark_id, 'ARK')

external link

In [ ]:
r_tx, est_gas = invoke_contract_experimental(w3,account,chain_id, dpid_service, 'add_externalLinks',dark_id, 'https://doi.org/10.1016/j.is.2021.101826') 

payload

In [ ]:
payload_data = { 'title' : 'Blockchain-based Privacy-Preserving Record Linkage: enhancing data privacy in an untrusted environment.' ,
                 'author' : 'Thiago Nóbrega',
                 'type' : 'Article'
            }


r_tx, est_gas = invoke_contract_experimental(w3,account,chain_id, dpid_service, 'set_payload',dark_id, str(payload_data)) 

In [ ]:
sys.getsizeof(str(payload_data)),sys.getsizeof(payload_data)